In [ ]:
%load_ext autoreload
%autoreload 2
#%reload_ext autoreload

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import Notebook.Generator as Generator 
import Notebook.model_definitions as model_definitions

### Open input file

In [ ]:

PATH= "Database\GamesDatabase.txt"
COLUMNS=["Player", "X", "Z", "VEL_X","VEL_Z","ROT","ANG_VEL_Y","ACC_X","ACC_Z","TILE","TILE_IND","X_RELATIVE","Z_RELATIVE","TIME"]
ColumnsToRead=["Player", "X", "Z", "VEL_X","VEL_Z","ROT","ANG_VEL_Y","ACC_X","ACC_Z","TILE","TILE_IND","X_RELATIVE","Z_RELATIVE","TIME","ACC","TURN","BREAK"]
DiscardColumns=["ACC","TURN","BREAK"]
DISCARD_COMMANDS=True

In [ ]:
SEQUENCE_LENGTH=50
BATCH_SIZE=512
FEATURES=7
FEATURES_PREDICTED=5
DISCARD=9

In [ ]:
Generator.SEQUENCE_LENGTH=SEQUENCE_LENGTH
Generator.FEATURES=FEATURES
Generator.DISCARD=DISCARD
Generator.COLUMNS=COLUMNS
train,val,test=Generator.single_care_dataframe(PATH,ColumnsToRead,DiscardColumns)

In [ ]:
def SubtractDF(df):
    return df.groupby("RACE",group_keys=False).apply(Generator.subtraction_columns)

def SubtractAndBuildDF(df):
    dfs=[]
    for i in range(len(df)):
        dfs.append(SubtractDF(df[i]))
        #dfs.append(df[i])
    united=dfs[0]
    for i in range(1,len(df)):
        united=pd.concat([united,dfs[i] ],ignore_index=True)
    return united

df_train= SubtractAndBuildDF(train)
df_val= SubtractAndBuildDF(val)
df_test= SubtractAndBuildDF(test)

In [ ]:
df_train.head()

In [ ]:
def check_nan_values(dataframe):
    nan_values = dataframe.isnull().sum()
    if nan_values.sum() == 0:
        print("No NaN values found in the DataFrame.")
    else:
        print("NaN values found in the DataFrame:")
        for col, count in nan_values.items():
            if count > 0:
                print(f"Column '{col}' has {count} NaN value(s) at indexes:")
                nan_indexes = dataframe[dataframe[col].isnull()].index.tolist()
                print(nan_indexes)
                return nan_indexes
                
ind=check_nan_values(df_train)
ind_1=check_nan_values(df_val)
ind_2=check_nan_values(df_test)

In [ ]:
gen_train=Generator.DataGenerator(BATCH_SIZE,df_train,100000)
gen_val=Generator.DataGenerator(BATCH_SIZE,df_val,100000)
gen_test=Generator.DataGenerator(BATCH_SIZE,df_test,100000)


### Model definition

#### Train and print

In [ ]:
model_definitions.cars=1
model_definitions.features=FEATURES
model_definitions.sequence_length=SEQUENCE_LENGTH
model_definitions.feat_pred=FEATURES_PREDICTED

### Behaviour AI Engine Model

LSTM & MLP Model

In [ ]:
lr =1e-4
units=36
cells=1
normalization=False
mlp_units=64
mlp_cells=0
dropout=0.4
name=f"LSTM{cells}_{units}_{lr}_{DISCARD}_MLP_{mlp_cells}_{mlp_units}_DROPOUT_{dropout}"
lstm_ph,history_lstm_ph=model_definitions.LSTM_Train(
    gen_train,
    gen_val,
    learning_rate=lr,
    patience=15,
    name=name,
    path=f"LSTM_MLP",
    epoch=100, 
    dropout=dropout,
    cells=cells,
    units=units,
    mlp_units=mlp_units,
    mlp_cells=mlp_cells,
    normalization=normalization,
    verbose=1
)
ev=lstm_ph.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")
#tf.saved_model.save(lstm_ph,f'saved_model/{name}')